# データ資産の作成方法

**参考URL**
- https://learn.microsoft.com/ja-jp/azure/machine-learning/how-to-create-data-assets?view=azureml-api-2
- https://github.com/yomaki/azure-ml-python-handson

**データ資産を作る目的**  
- バージョン管理が容易
- 再現性: バージョンごとに、変更・削除ができないため、データ資産を取り込んでトレーニングジョブやパイプラインを再現できる。データを変更したい場合は、バージョンを上げて新しく作成する。
- トレーサビリティ: 誰がいつ作ったかを確認できる。また、モデルごとにどのデータを使用してトレーニングしたかを確認できる。

**概要**  
データ資産は、Webブラウザーのブックマークに似ている。最も頻繁に使用されるデータを示す長いストレージパス(URI)を覚える代わりに、データ資産を作成して、フレンドリ名を使ってその資産にアクセスできる。
例)Azure Blob Storageのデータにアクセスすることを考える。  
- 一般的な接続方法<br>
  blob storageデータのURI(`abfss://<account_name>.blob.core.windows.net/<container_name>/<folder_name>/<file>`)を直接記述して接続する。
  
- データ資産を活用する方法<br>
  blob_storageデータのURIから、Azure ML上にデータ資産を作成する。  
  データロードするときは、このデータ資産の短い名前でアクセスする。


**データ資産の種類について**  
データ資産を作成する時には、種類を設定する必要があり、次の3種類をサポートしている。
|Type|API|ユースケース|
|--- | --| ---      |
| 1つのファイル | `uri_file` | Azure Storage上の個別のファイルを読み取る.|
| フォルダー| `uri_folder` | Parquet/CsvファイルのフォルダーをPandas/Sparkに読み取る。<br>フォルダーに格納されている非構造データ(画像、テキスト、オーディオなど)を読み取る。
| Table | `mltable` | テーブルを使用したAutoML. <br> 複数の保存場所にまたがって存在する非構造データを読み取る。 |

**データ資産のタグづけについて**  
データ資産についてタグづけがサポートされており、追加のメタデータをキーと値として保存できる。
以下のユースケースがあります。
- データ品質の説明. 例えば、`data_type:raw`(生データ)、`data_type:preprocessed`(前処理済み)、`data_type:enriched`: エンリッチ済み(画像と共にキャプションが付与されている場合など)
- データの効率的な検索とフィルター処理
- 機密性の高いデータの管理。(`sensitivity:High`, `sensitivity:Low`)


In [12]:
!az login

A web browser has been opened at https://login.microsoftonline.com/organizations/oauth2/v2.0/authorize. Please continue the login in the web browser. If no web browser is available or if the web browser fails to open, use device code flow with `az login --use-device-code`.
The following tenants don't contain accessible subscriptions. Use 'az login --allow-no-subscriptions' to have tenant level access.
ebff0699-bfb6-48c6-be53-efe17fad2453 '既定のディレクトリ'
[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "b67095c9-a754-4c1d-9bdc-1737fea9fcd2",
    "id": "a74c5e64-03e5-4eb2-91dc-1280992ff086",
    "isDefault": true,
    "managedByTenants": [],
    "name": "サブスクリプション 1",
    "state": "Enabled",
    "tenantId": "b67095c9-a754-4c1d-9bdc-1737fea9fcd2",
    "user": {
      "name": "sitomi0216@icloud.com",
      "type": "user"
    }
  },
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "b67095c9-a754-4c1d-9bdc-1737fea9fcd2",
    "id": "4946c281-57db-44b4-a10a-cf4b965cbf82",
    "isDefa

In [17]:
# クラインアントを認証する
import os, sys 
from azure.ai.ml import MLClient 
from azure.identity import DefaultAzureCredential 
from azure.core.exceptions import HttpResponseError
from dotenv import load_dotenv
load_dotenv(verbose=True) 

_subscription_id = os.environ['SUBSCRIPTION_ID']
_resource_group = os.environ['RESOURCE_GROUP']
_aml_workspace_name = os.environ['AML_WORKSPACE_NAME']

# このクライアントを作っても、ワークスペースに接続はされない。
# 初めて呼び出す必要が生じた場合に初めて接続されます。
ml_client = MLClient(
    DefaultAzureCredential(),
    subscription_id=_subscription_id,
    resource_group_name=_resource_group,
    workspace_name=_aml_workspace_name
)

**クラウドストレージにデータをアップロード**  
Azure MLでは、クラウド内のストレージの場所を指すURI(Uniform Resource Identifier)を使用します。
データ資産を作成すると、データソースの場所への”参照”とそのメタデータのコピーが作成される。
データは既存の場所に残るので、追加のストレージコストが発生せず、データソースの整合性が維持される。

データ資産を作成するごとに一意のバージョンが必要です。バージョンがすでに存在する場合は、エラーが発生します。
versionパラメータを省略した場合は、1から始まりそこからインクリメントされて生成される。


In [18]:
from azure.ai.ml.entities import Data 
from azure.ai.ml.constants import AssetTypes


data_path = os.path.join(os.pardir, 'data', 'diabetes.csv')
v1 = 'initial'

my_data = Data(
    name='diabetes',
    version=v1,
    description="Diabetes Data",
    path=data_path,
    type=AssetTypes.URI_FILE
)

## データが存在しない場合に作成する。
try:
    ml_client.data.create_or_update(my_data)
    print('done')
except HttpResponseError as e:
    print('data is already exist.')

data is already exist.


## データにアクセスする

In [24]:
import pandas as pd 

df = pd.read_csv(f"azureml://subscriptions/{_subscription_id}/resourcegroups/{_resource_group}/workspaces/{_aml_workspace_name}/datastores/workspaceblobstore/paths/LocalUpload/d52d15a0d7d1e95b90a03f146099424a/diabetes.csv")

df

ImportError: Missing optional dependency 'fsspec'.  Use pip or conda to install fsspec.